<a href="https://colab.research.google.com/github/kgautam2103/zoom_participant_list_public/blob/main/Realization_program_ndays_zoom_report_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from google.colab import drive
drive.mount("mnt",force_remount=True)
%run '/content/mnt/MyDrive/Colab Notebooks/zoom_participant_list_utility_functions.ipynb'


Mounted at mnt


In [ ]:
import requests
import random
import json
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import gspread
import re
from pandas import ExcelWriter
from google.colab import files
#from gspread_dataframe import set_with_dataframe
from pandas import json_normalize
from urllib.parse import urlencode
#!pip install python-Levenshtein
#!pip install fuzzywuzzy
#from fuzzywuzzy import process
!pip install gspread-pandas
from gspread_pandas import Spread, Client


In [ ]:
report_date= input("enter the report datetime(UTC) like 2021-07-11T02 : ")
report_datetime = report_date[:-3]
event_name = input("enter the event name like CHN_Day_1 : ")

meetingID = '9181716151'
page_size ='300'
base_url = 'https://api.zoom.us/v2/'
past_meetings = base_url+'/past_meetings/'


#to get result for the last held meeting
report_meetings = base_url+'report/meetings/'
get_meeting_url = report_meetings+meetingID

get_meeting_url = past_meetings+meetingID+'/instances'
get_user_url = base_url+'users/'

#auth token for zoom account admin user
auth_token = ''

authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

meeting_id_response = requests.get(get_meeting_url,headers=auth_headers)


if meeting_id_response.status_code == 200:

  for x in meeting_id_response.json()['meetings']:
    if report_date in str(x['start_time']):
      meeting_uid = str(x['uuid'])
      print(x)

enter the report datetime(UTC) like 2021-07-11T02 : 2022-01-12T11
enter the event name like CHN_Day_1 : CHN_Day_2
{'uuid': 'bzServAwRduBzBSS9TalCg==', 'start_time': '2022-01-12T11:52:04Z'}


In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/17qtqgcYSLrlOs3YA0Lzn_Fk710PFL9qUO1wtdyBoCpo/')

worksheet = wb.add_worksheet(title=event_name, rows="1000", cols="20")

combined_sheet = wb.worksheet("Combined")
combined_df = pd.DataFrame(combined_sheet.get_all_records())


if meeting_uid != "":
  meeting_encoded_uid = double_encode_uuid(meeting_uid)
  participants_list = get_participants_list(meeting_encoded_uid)

if len(participants_list)>0:
  participants_df = get_participants_df(participants_list,event_name,[],[],[])

if combined_df.empty:
  print("its empty")
  combined_sheet.update([participants_df.columns.values.tolist()] + participants_df.values.tolist())
else:
  final_df = pd.concat([combined_df, participants_df], axis=0)
  combined_sheet.update([final_df.columns.values.tolist()] + final_df.values.tolist())

worksheet.update([participants_df.columns.values.tolist()] + participants_df.values.tolist())


{'spreadsheetId': '17qtqgcYSLrlOs3YA0Lzn_Fk710PFL9qUO1wtdyBoCpo',
 'updatedCells': 760,
 'updatedColumns': 8,
 'updatedRange': 'CHN_Day_2!A1:H95',
 'updatedRows': 95}